In [5]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

auth_url = "https://api.petfinder.com/v2/oauth2/token"

data = {
    "grant_type": "client_credentials",
    "client_id": os.getenv("API_KEY"),
    "client_secret": os.getenv("API_SECRET")
}

response = requests.post(auth_url, data=data)

if response.status_code != 200:
    raise RuntimeError(f"Error {response.status_code}: {response.text}")

bearer = response.json()

In [ ]:
def get_photos(type, minimum_amount=1000):
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {bearer['access_token']}"}
    page = 1
    photos = []

    while len(photos) < minimum_amount:
        response = requests.get(
            url, headers=headers, params={"type": type, "limit": 100, "page": page}
        )
        page += 1

        if response.status_code != 200:
            raise RuntimeError(f"Error {response.status_code}: {response.text}")

        data = response.json()
        for animal in data["animals"]:
            for photo in animal["photos"]:
                photos.append(photo["medium"])

    return photos

In [ ]:
def download_photos(type):
    folder_path = os.path.join(os.getcwd(), "data", f"{type}s")
    os.makedirs(folder_path, exist_ok=True)

    photos = get_photos(type)

    for i, url in enumerate(photos):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            file_name = f"{i}.jpeg"
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, "wb") as f:
                f.write(response.content)

        except Exception as e:
            print(f"Failed to download {url}: {e}")

In [ ]:
download_photos("dog")
download_photos("cat")

After running this script, manually create a train, validation and test folder, then move part of the photos in each